파이토치(PyTorch)의 nn.Embedding()은 단어 각각에 대해 정수가 맵핑된 입력에 대해서 임베딩 작업을 수행할 수 있게 해줍니다.

### 실제 RNN 은닉층을 추가하는 코드.
nn.RNN(input_size, hidden_size, batch_first=True)

텍스트 분류 관점에서 앞서 배운 RNN 코드의 timesteps와 input_dim, 그리고 hidden_size를 해석해보면 다음과 같습니다. (위의 코드에서는 바닐라 RNN을 사용했지만, RNN의 변형인 LSTM이나 GRU도 아래의 사항은 동일합니다.)

hidden_size = 출력의 크기(output_dim).  
timesteps = 시점의 수 = 각 문서에서의 단어 수.  
input_size = 입력의 크기 = 각 단어의 벡터 표현의 차원 수.  